# "Machine learning con Python"
> "Guía de referencia rápidda de machine learning con Python"

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [python, kaggle, ML]
- image: images/python.png
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

> tip: Puedes ver este post en GitHub o ejecutarlo en Binder o Google Colab, pulsa el icono.

Fuentes:<br>
[Cursos Machine Learning - Kaggle](https://www.kaggle.com/learn)<br>


## 1 - Recoger los datos

## 2 - Preparar los datos

### 2.1 - Exploración de los datos

In [ ]:
import pandas as pd

datos = pd.read_csv(ruta_archivo) # cargar datos desde tabla en archivo csv
datos.describe()                  # imprime resumen de los datos
datos.columns                     # imprime lista de las columnas
datos.head()                      # imprime lista de las 5 primeras muestras de la tabla

datos_filtrados = datos.dropna(axis=0) # elimina datos na (no available - no disponibles)

In [ ]:
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True) # Eliminar filas sin datos destino, 

# elegir la columna que deseamos estimar en nuestro modelo
y = X_full.SalePrice

# elegir las columnas que se pasarán al modelo
# opción 1: quitar la columna destino
X_full.drop(['SalePrice'], axis=1, inplace=True)

# opción 2: elegir sólo algunas columnas
datos_features = ['Rooms', 'Bathroom','Lattitude', 'Longtitude'] 
X = datos[datos_features]

### 2.2- Valores que faltan

In [ ]:
missing_val_count_by_column = (X_train.isnull().sum()) # Número de valores faltantes en cada columna de datos de entrenamiento 

print(missing_val_count_by_column[missing_val_count_by_column > 0])

#### 2.2.1 - Eliminar columna
Para el caso en falten la mayoría de datos de una variable. Sino se puede perder información importante.

In [ ]:
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]       # recopilar columnas con valores que faltan

# eliminar columnas con valores que faltan
reduced_X_train = X_train.drop(cols_with_missing, axis=1)  
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

#### 2.2.2 - Imputación
Rellenar los valores faltantes con otro valor, por ejemplo: la media.

In [ ]:
from sklearn.impute import SimpleImputer

# Imputación
my_imputer = SimpleImputer(strategy='median')

imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train), index=range(1, X_train.shape[0] + 1),
                          columns=range(1, X_train.shape[1] + 1))  
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid), index=range(1, X_valid.shape[0] + 1),
                          columns=range(1, X_valid.shape[1] + 1))  

# Imputación quitó los nombres de columnas, volver a ponerlos 
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

#### 2.2.3 - Extensión de Imputación
Añadiendo otra columna que informe si faltaba o no el valor.

In [ ]:
X_train_plus = X_train.copy() # Hacer una copia para evitar cambiar los datos originales (al imputar) 
X_valid_plus = X_valid.copy()

# Hacer nuevas columnas que indiquen lo que se imputará. 
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputación
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputación quitó los nombres de columnas, volver a ponerlos 
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns


### 2.3.- Variables categóricas
Toman un número limitado de valores. Necesitan ser preprocesados antes de usarlos en los modelos.

In [ ]:
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]
# para obtener lista de variables categóricas 

print(object_cols)

In [ ]:
good_label_cols = [col for col in object_cols if   # Columnas que se pueden codificar con etiquetas de forma segura 
                   set(X_train[col]) == set(X_valid[col])]
        
# Columnas problemáticas que se eliminarán del conjunto de datos
bad_label_cols = list(set(object_cols)-set(good_label_cols))

# Aplicar codificador de etiquetas 
label_encoder = LabelEncoder()
for col in good_label_cols:
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_valid[col] = label_encoder.transform(X_valid[col])

In [ ]:
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols)) 
# Obtener el número de entradas únicas en cada columna con datos categóricos 
d = dict(zip(object_cols, object_nunique))

# Imprimirc el número de entradas únicas por columna, en orden ascendente 
sorted(d.items(), key=lambda x: x[1])

#### 2.3.1- Eliminar columna
Sólo es buena opción si la columna no tiene información útil.

In [ ]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

#### 2.3.2- Codificación ordinal
Asigna un entero a cada valor único. Asume un orden en las categorías. Variables ordinales.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# Hacer una copia para evitar cambiar los datos originales 
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Aplicar codificador ordinal a cada columna con datos categóricos 
ordinal_encoder = OrdinalEncoder()
label_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
label_X_valid[object_cols] = ordinal_encoder.transform(X_valid[object_cols])

# De esta forma ha asigando valores aleatorios a las categorías. Asignar un orden puede mejorar el modelo.

#### 2.3.3- Codificación *One-Hot*
Crea una columna por cada valor único e indica la presencia o no de ese valor. No asume un orden en las categorías. Variables nominales. No funciona bien si hay muchas categorias. Usar si hay 15 como máximo.

* handle_unknown='ignore': para evitar errores cuando los datos de validación contienen clases que no están representadas en los datos de entrenamiento 
* sparse=False: asegura que las columnas codificadas se devuelvan como una matriz densa (en lugar de una matriz dispersa)

Suele ser la mejor aproximación.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Columnas que se codificarán one-hot (como máximo 10 categorias)
low_cardinality_cols = [col for col in object_cols if X_train[col].nunique() < 10]

# Columnas que se eliminarán
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))


# Aplicar codificador one-hot a cada columna con datos categóricos
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[low_cardinality_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[low_cardinality_cols]))

# codificador one-hot elimina; ponerlo de nuevo 
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Eliminar columnas categóricas (se reemplazarán con codificación one-hot) 
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

#  añadir columnas codificadas one-hot a variables numéricas 
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)


In [ ]:
from sklearn.preprocessing import OneHotEncoder


### Separar los datos para entrenar y validar

In [ ]:
X = X_full.select_dtypes(exclude=['object']) # To keep things simple, we'll use only numerical predictors
X_test = X_test_full.select_dtypes(exclude=['object'])

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0) 

## 3- Seleccionar el modelo

In [ ]:
from sklearn.tree import DecisionTreeRegressor  # RandomForestRegressor

modelo = DecisionTreeRegressor(random_state=1)    # escogemos el modelo Árbol de decisión


## 4- Entrenar el modelo

In [ ]:
modelo.fit(train_X, train_y)                      # se entrena el modelo 

## 5- Evaluar el modelo

In [ ]:
val_predicciones = modelo.predict(val_X)          # se hace predicción
mean_absolute_error(val_y, val_predicciones)      # se evalua el modelo

### Underfitting and Overfitting

   * *Underfitting*: no encuentra bien patrones relevantes, no predice bien
   * *Overfitting*:  se ajusta demasiado a los datos de entrenamiento, no generaliza bien

## 6- Ajustar los parámetros

## 7- Generar la predicciones

In [ ]:
final_X_test = pd.DataFrame(final_imputer.transform(X_test))   # usar total de datos transformados

# Sacar las predicciones con los datos para test 
preds_test = model.predict(final_X_test)


# Guardar datos a archivo
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)

## Otros puntos a tener en cuenta

### Pipelines (canalizaciones)
Para agrupar partes de código. Ventajas:
- Código limpio
- Menos errores
- Más fácil de llevar a producción
- Más opciones para validar el modelo

#### 1- Definir pasos del preprocesado

In [1]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocesamiento de datos numéricos 
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocesamiento de datos categóricos
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Paquete de prepocesado
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

NameError: name 'numerical_cols' is not defined

#### 2- Definir modelo

In [2]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

#### 3- Crear y evaluar el preprocesado

In [ ]:
from sklearn.metrics import mean_absolute_error

# Agrupar código de preprocesamiento y modelado en una canalización 
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Procesamiento previo de datos de entrenamiento, modelo de ajuste
my_pipeline.fit(X_train, y_train)

# Procesamiento previo de datos de validación, obtención de predicciones
preds = my_pipeline.predict(X_valid)

# Evaluar el modelo
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

### Cross-Validation
Divide los datos en subconjuntos, realiza el entrenamiento y validación alternando los subconjuntos y calcula la media al final.
Para conjuntos de datos grandes es posible que no sea necesario. Si Cross-validation da resultados similares en cada subconjunto no es necesario hacerlo.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

my_pipeline = Pipeline(steps=[('preprocessor', SimpleImputer()),
                              ('model', RandomForestRegressor(n_estimators=50,
                                                              random_state=0))
                             ])

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("MAE scores:\n", scores)

### Método XGBoost
Consiste en un ciclo donde se van añadiendo modelos reduciéndose el error paulatinamente.
<br> Usar con datos tabulados, no imagenes o video...

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

my_model = XGBRegressor()
my_model.fit(X_train, y_train)

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

**n_estimators** = nº ciclos = nº modelos a añadir. Suele  estar entre 100-1000 (depende mucho del ratio de aprendizaje).
<br>
- muy bajo -> underfitting
- muy alto -> overfitting

**early_stopping_rounds** encuentra automáticamente el valor óptimo de 'n_estimators'. 
Para el entrenamiento cuando el valor de validación deja de mejorar. Valor adecuado=5, para cuando lleva 5 ciclos empeorando la validación.

**learning_rate** = ratio de aprendizaje. Por defecto 0.1.  
Por lo general, es mejor nº alto de 'n_estimators' y  bajo de 'learning_rate'

**n_jobs** = para ejecución en paralelo de grandes conjuntos de datos = nº nucleos del ordenador

In [ ]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

### Data leakage (fuga de datos)


Ocurre cuando el conjunto de datos tiene información de la variable de destino.
Produce alta precisión en el entrenamiento, pero muy baja en la predicciones reales.

Tipos:
- **target leakage**: cuando se incluyen datos que tienen información posterior a lo que se desea predecir
- **train-test contamination**: ocurre cuando se mezclan los datos de entrenamiento y los de validación a la hora de entrenar el modelo. 